In [2]:
import os, csv, time, datetime, random, json, warnings
import gensim
import pandas as pd
import numpy as np
import tensorflow as tf
from collections import Counter
from math import sqrt
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score

warnings.filterwarnings("ignore")

d:\Users\123\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\Users\123\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\Users\123\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\Users\123\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type

In [3]:
directory_path = "\\".join(os.getcwd().split("\\")[:-1]) + "\\data"
os.path.exists(directory_path)

True

In [4]:
class TrainingConfig(object):
    epochs = 10
    evaluateEvery = 100
    checkpointEvery = 100
    learningRate = 0.001


class ModelConfig(object):
    embeddingSize = 200
    
    # LSTM 的神经元个数
    hiddenSizes = [128]
    dropoutKeepProb = 0.5
    l2RegLambda = 0.0
    
    outputSize = 128

    
class Config(object):
    sequenceLength = 200
    batchSize = 128
    dataSource =directory_path + "\\preProcess\\labeledTrain.csv"
    stopWordSource = directory_path + "\\english"
    
    # 二分类设置为 1 ，多分类设置为其他数值
    numClasses = 1
    #训练集比例
    rate = 0.8
    training = TrainingConfig()
    model = ModelConfig

config = Config()

In [8]:
class Dataset(object):
    def __init__(self, config):
        self.config = config
        self._dataSource = config.dataSource
        self._stopWordSource = config.stopWordSource
        self._sequenceLength = config.sequenceLength
        self._embeddingSize = config.model.embeddingSize
        self._batchSize = config.batchSize
        self._rate = config.rate
        
        self._stopWordDict = dict()
        self.trainReviews = []
        self.trainLabels= []
        self.evalReviews = []
        self.evalLabels = []
        self.wordEmbedding = None
        self.labelList = []
        
    
    def _readData(self, file_path):
        df = pd.read_csv(file_path)
        if self.config.numClasses == 1:
            labels = df["sentiment"].tolist()
        elif self.config.numClasses > 1:
            labels = df["rate"].tolist()
        
        review = df["review"].tolist()
        reviews = [line.strip().split() for line in review]
        
        return reviews, labels
    
    
    def _labelToIndex(self, labels, label2idx):
        labelIds = [label2idx[label] for label in labels]
#         print("the labelIds are: ", labelIds)
        return labelIds
    
    
    def _wordToIndex(self, reviews, word2idx):
        reviewIds = [[word2idx.get(item, word2idx["UNK"]) for item in review] for review in reviews]
#         print("the review Ids are: ", reviewIds)
        return reviewIds
    
    
    def _genTrainEvalData(self, x, y, word2idx, rate):
        reviews = []
        for review in x:
            if len(review) > self._sequenceLength:
                reviews.append(review[:self._sequenceLength])
            else:
                reviews.append(review + [word2idx["PAD"]] * (self._sequenceLength - len(review)))
        
        trainIndex = int(len(x) * rate)
        trainReviews = np.asarray(reviews[:trainIndex], dtype="int64")
        trainLabels = np.array(y[:trainIndex], dtype="float32")
        evalReviews = np.asarray(reviews[trainIndex:], dtype="int64")
        evalLabels = np.array(y[trainIndex:], dtype="float32")
        
        return trainReviews, trainLabels, evalReviews, evalLabels
    
    
    def _genVocabulary(self, reviews, labels):
        all_words = [word for review in reviews for word in review]
        subWords = [word for word in all_words if word not in self.stopWordDict]
    
        wordCount = Counter(subWords)
        sortedWordCount = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
        words = [item[0] for item in sortedWordCount if item[1] >= 5]
        
        vocab, wordEmbedding = self._getWordEmbedding(words)
        self.wordEmbedding = wordEmbedding
        word2idx = dict(zip(vocab, list(range(len(vocab)))))
        
        uniqueLabel = list(set(labels))
        label2idx = dict(zip(uniqueLabel, list(range(len(uniqueLabel)))))
        self.labelList = list(range(len(uniqueLabel)))
        
#         print("teh label list is: ", self.labelList)
        
        with open(directory_path + "\\wordJson\\word2idx.json", "w", encoding="utf-8") as f:
            json.dump(word2idx, f)
        
        with open(directory_path + "\\wordJson\\label2idx.json", "w", encoding="utf-8") as f:
            json.dump(label2idx, f)
    
        return word2idx, label2idx
    
    
    def _getWordEmbedding(self, words):
        wordVec = gensim.models.KeyedVectors.load_word2vec_format(directory_path + "\\word2vec\\word2Vec.bin", binary=True)
        vocab = []
        wordEmbedding = []
        
        vocab.append("PAD")
        vocab.append("UNK")
        wordEmbedding.append(np.zeros(self._embeddingSize))
        wordEmbedding.append(np.random.randn(self._embeddingSize))
        
        for word in words:
            try:
                vector = wordVec.wv[word]
                vocab.append(word)
                wordEmbedding.append(vector)
            except:
                print("{} is not exist".format(word))
        
        return vocab, np.array(wordEmbedding)
    
    
    def _readStopWord(self, stopWordPath):
        with open(stopWordPath, "r") as f:
            stopWords = f.read()
            stopWordList = stopWords.splitlines()
            self.stopWordDict = dict(zip(stopWordList, list(range(len(stopWordList)))))
    
    
    def dataGen(self):
        self._readStopWord(self._stopWordSource)
        reviews, labels = self._readData(self._dataSource)
        word2idx, label2idx = self._genVocabulary(reviews, labels)
        
        labelIds = self._labelToIndex(labels, label2idx)
        reviewIds = self._wordToIndex(reviews, word2idx)
        
        trainReviews, trainLabels, evalReviews, evalLabels = self._genTrainEvalData(reviewIds, 
                                                                                   labelIds, 
                                                                                   word2idx, 
                                                                                   self._rate)
        self.trainReviews = trainReviews
        self.trainLabels = trainLabels
        self.evalReviews = evalReviews
        self.evalLabels = evalLabels
        

    
data = Dataset(config)
data.dataGen()

the word2idx is:  {'puerile': 11255, 'debate': 5736, 'fakes': 21845, 'transparent': 7673, 'zombie-like': 25255, 'nu': 28604, 'searchers': 19111, 'degrade': 21469, 'bloodshed': 9115, 'effectsthe': 20192, 'courses': 18866, 'electronic': 7861, 'againand': 22655, 'bart': 8623, 'titanics': 30344, 'compiled': 19390, 'celtic': 17209, 'reused': 22207, 'blackmails': 19645, 'tawdry': 15219, 'manservant': 31977, 'unwashed': 20193, 'ad-libbed': 21470, 'surefire': 24042, 'huntingdon': 30345, 'novak': 5648, 'timeit': 23061, 'foreshadowed': 29454, 'beban': 24362, 'this!': 3821, 'financing': 16053, 'writerdirector': 3223, 'ended': 934, 'gamera': 6469, 'coworker': 21846, 'aip': 31540, 'gentile': 28605, 'hepburn': 6723, 'rarity': 10869, 'sob': 16210, 'parents': 677, 'stan': 3235, 'subs': 16211, 'per': 3215, 'comprehend': 6605, 'shutting': 16212, 'maker': 3840, 'connects': 9769, 'petrol': 20815, 'blended': 10896, '2-d': 29455, 'andress': 30346, 'archival': 16797, 'plot-line': 15549, 'smarmy': 9091, 'ryke

In [10]:
def nextBatch(x, y, batchSize):
    perm = np.arange(len(x))
    np.random.shuffle(perm)
    x = x[perm]
    y = y[perm]
    numBatches = len(x) // batchSize
    for index in range(numBatches):
        start = index * batchSize
        end = start + batchSize
        batchX = np.array(x[start: end], dtype="int64")
        batchY = np.array(y[start: end], dtype="float32")
        
        yield batchX, batchY

In [29]:
# model define
# 1、利用 bi-LSTM 获取上下文信息
# 2、将 bi-LSTM 获取到的隐藏层输出，和词向量拼接 [fw_out; word_embedding; bw_out]
# 3、将step 2 所得到的 word represention map to lower-dims
# 4、hidden-size 上每个位置上的值，均取时间步上最大的值，类似于 max-pool
# 5、softmax

class RCNN(object):
    def __init__(self, config, wordEmbedding):
        self.inputX = tf.placeholder(tf.int32, [None, config.sequenceLength], name="inputX")
        self.inputY = tf.placeholder(tf.int32, [None], name="inputY")
        self.dropoutKeepProb = tf.placeholder(tf.float32, name="dropoutKeepProb")
        
        l2Loss = tf.constant(0.0)
        with tf.name_scope("embedding"):
            self.W = tf.Variable(tf.cast(wordEmbedding, dtype=tf.float32, name="word2vec"), name="W")
            self.embeddedWords = tf.nn.embedding_lookup(self.W, self.inputX)
            self.embeddedWords_ = self.embeddedWords
            
        # 定义双向的 bi-LSTM model
        with tf.name_scope("Bi-LSTM"):
            for index, hiddenSize in enumerate(config.model.hiddenSizes):
                with tf.name_scope("Bi-LSTM" + str(index)):
                    # 定义前向 LSTM 结构
                    lstmFwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, 
                                                                                       state_is_tuple=True),
                                                               output_keep_prob=self.dropoutKeepProb)
                    
                    # 定义反向的 LSTM 结构
                    lstmBwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, 
                                                                                      state_is_tuple=True), 
                                                              output_keep_prob=self.dropoutKeepProb)
                    
                    # 采用动态 RNN ，可以动态地输入序列的长度，如若没有输入，则取序列全长
                    # outputs 是一个元组 (output_fw, output_bw), 其中两个元素的维度均是 [batch_size, max_time, hidden_size]
                    # fw 和 bw 的 hidden_size 一样
                    # self.current_state 是最终状态，二元组 (state_fw, state_bw), state_fw = [batch_size, s]；s 表示的是一个元组 (h, c)
                    
                    outputs_, self.current_state = tf.nn.bidirectional_dynamic_rnn(lstmFwCell, lstmBwCell, 
                                                                                  self.embeddedWords_, 
                                                                                   dtype=tf.float32, 
                                                                                  scope="bi-lstm" + str(index))
                    
                    # 针对 outputs_ 中的 fw 和 bw 的结果进行拼接
                    # [batch_size, time_step, hidden_size * 2] 传入到下一层 bi-LSTM 中
                    self.embeddedWords_ = tf.concat(outputs_, 2)
        
        
        # 针对最后一层的"bi-LSTM"进行分割
        fwOutput, bwOutput = tf.split(self.embeddedWords_, 2, -1)
        
        print("the shape of fwOutput is: ", tf.shape(fwOutput))
        print("the shape of bwOutput is: ", tf.shape(bwOutput))
        
        with tf.name_scope("context"):
            shape = [tf.shape(fwOutput)[0], 1, tf.shape(fwOutput)[2]]
            self.contextLeft = tf.concat([tf.zeros(shape), fwOutput[:, :-1]], axis=1, name="contextLeft")
            self.contextRight = tf.concat([bwOutput[:, 1:], tf.zeros(shape)], axis=1, name="contextRight")
        
        
        # 将前向、后向的输出和原始的词向量拼接在一起，或得到最终的词向量表示
        with tf.name_scope("wordRepresentation"):
            self.wordRepre = tf.concat([self.contextLeft, self.embeddedWords, self.contextRight], axis=2)
            wordSize = config.model.hiddenSizes[-1] * 2 + config.model.embeddingSize
        
        
        with tf.name_scope("textRepresentation"):
            outputSize = config.model.outputSize
            textW = tf.Variable(tf.random_uniform([wordSize, outputSize], -1.0, 1.0), name="W2")
            textB = tf.Variable(tf.constant(0.1, shape=[outputSize]), name="b2")
            
            # tf.einsum 可以指定维度进行消除运算
            self.textRepre = tf.tanh(tf.einsum("aij, jk -> aik", self.wordRepre, textW) + textB)
        
        
        # max-pool 操作，同时消除掉时间步 step-time
        output = tf.reduce_max(self.textRepre, axis=1)
        
        
        # 全连接层
        with tf.name_scope("output"):
            outputW =tf.get_variable("outputW", shape=[outputSize, config.numClasses], 
                                     initializer=tf.contrib.layers.xavier_initializer())
            outputB = tf.Variable(tf.constant(0.1, shape=[config.numClasses]), name="outputB")
            
            l2Loss += tf.nn.l2_loss(outputW)
            l2Loss += tf.nn.l2_loss(outputB)
            self.logits = tf.nn.xw_plus_b(output, outputW, outputB, name="logits")
            
            if config.numClasses == 1:
                self.predictions = tf.cast(tf.greater_equal(self.logits, 0.0), tf.float32, name="predictions")
            elif config.numClasses > 1:
                self.predictions = tf.argmax(self.logits, axis=-1, name="predictions")
            
        
        # 计算二元交叉熵损失函数
        with tf.name_scope("loss"):
            if config.numClasses == 1:
                losses = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.logits, 
                                                                labels=tf.cast(tf.reshape(self.inputY, [-1, 1]), 
                                                                              dtype=tf.float32))
            elif config.numClasses > 1:
                losses = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=self.logits, labels=self.inputY)
            
            self.loss = tf.reduce_mean(losses) + config.model.l2RegLambda * l2Loss
            
        
        
        

In [30]:
def mean(item):
    res = sum(item) / len(item) if len(item) > 0 else 0
    return res

def accuracy(pred_y, true_y):
    if isinstance(pred_y[0], list):
        pred_y = [item[0] for item in pred_y]
    
    corr = 0
    for index in range(len(pred_y)):
        if pred_y[index] == true_y[index]:
            corr += 1
    
    acc = corr / len(pred_y) if len(pred_y) > 0 else 0
    
    return acc


def binary_precision(pred_y, true_y, positive=1):
    """
    :param positive: 正例的索引表示
    """
    corr = 0
    pred_corr = 0
    for index in range(len(pred_y)):
        if pred_y[index] == positive:
            pred_corr += 1
            if pred_y[index] == true_y[index]:
                corr += 1
    
    prec = corr / pred_corr if pred_corr > 0 else 0
    
    return prec


def binary_recall(pred_y, true_y, positive=1):
    corr = 0
    true_corr = 0
    for index in range(len(pred_y)):
        if true_y[index] == positive:
            true_corr += 1
            if true_y[index] == pred_y[index]:
                corr += 1
    
    rec = corr / true_corr if true_corr > 0 else 0
    
    return rec



def binary_f_beta(pred_y, true_y, beta=1.0, positive=1):
    precision = binary_precision(pred_y, true_y, positive)
    recall = binary_recall(pred_y, true_y, positive)
    try:
        f_b = (1 + beta * beta) * precision * recall / (beta * beta * precision + recall)
    except:
        f_b = 0
    
    return f_b



def multi_precision(pred_y, true_y, labels):
    if isinstance(pred_y[0], list):
        pred_y = [item[0] for item in pred_y]
    
    precisions = [binary_precision(pred_y, true_y, label) for label in labels]
    prec = mean(precisions)
    return prec


def multi_recall(pred_y, true_y, labels):
    if isinstance(pred_y[0], list):
        pred_y = [item[0] for item in pred_y]
    
    recalls = [binary_recall(pred_y, true_y, label) for label in labels]
    rec = mean(recalls)
    return rec


def multi_f_beta(pred_y, true_y, labels, beta=1.0):
    if isinstance(pred_y[0], list):
        pred_y = [item[0] for item in pred_y]
    
    f_beats =[binary_f_bate(pred_y, true_y, beta, label) for label in labels]
    f_beta = mean(f_betas)
    return f_beta



def get_binary_metrics(pred_y, true_y, f_beta=1.0):
    acc = accuracy(pred_y, true_y)
    recall = binary_recall(pred_y, true_y)
    precision = binary_precision(pred_y, true_y)
    f_beta = binary_f_beta(pred_y, true_y, f_beta)
    
    return acc, recall, precision, f_beta



def get_multi_metrics(pred_y, true_y, labels, f_beta=1.0):
    acc = accuracy(pred_y, true_y)
    recall = mulyi_recall(pred_y, true_y, labels)
    precision = multi_precision(pred_y, true_y, labels)
    f_beta = multi_f_beta(pred_y, true_y, labels, f_beta)
    
    return acc, recall, precision, f_beta




In [40]:
trainReviews = data.trainReviews
trainLabels = data.trainLabels
evalReviews = data.evalReviews
evalLabels = data.evalLabels

wordEmbedding = data.wordEmbedding
labelList = data.labelList

with tf.Graph().as_default():
    session_config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    sess = tf.Session(config=session_config)
    
    with sess.as_default():
        lstm = RCNN(config, wordEmbedding)
        globalStep = tf.Variable(0, name="globalStep", trainable=False)
        optimizer = tf.train.AdamOptimizer(config.training.learningRate)
        gradsAndVars = optimizer.compute_gradients(lstm.loss)
        trainOp = optimizer.apply_gradients(gradsAndVars, global_step=globalStep)
        
        
        # 用summary绘制tensorBoard
        gradSummaries = []
        for g, v in gradsAndVars:
            if g is not None:
                tf.summary.histogram("{}/grad/hist".format(v.name), g)
                tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
        
        outDir = os.path.abspath(os.path.join(os.path.curdir, "summarys"))
        print("Writing to {}\n".format(outDir))
        
        lossSummary = tf.summary.scalar("loss", lstm.loss)
        summaryOp = tf.summary.merge_all()
        
        trainSummaryDir = os.path.join(outDir, "train")
        trainSummaryWriter = tf.summary.FileWriter(trainSummaryDir, sess.graph)
        
        evalSummaryDir = os.path.join(outDir, "eval")
        evalSummaryWriter = tf.summary.FileWriter(evalSummaryDir, sess.graph)
        
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=5)
        savedModelPath = "\\".join(directory_path.split("\\")[:-1]) + "\\model\\textCNN\\savedModel"
        if os.path.exists(savedModelPath):
#             os.rmdir(savedModelPath)
            import shutil
            shutil.rmtree(savedModelPath)
            
        
        builder = tf.saved_model.builder.SavedModelBuilder(savedModelPath)
        sess.run(tf.global_variables_initializer())
        
        
        def trainStep(batchX, batchY):
            feed_dict = {
                lstm.inputX: batchX, 
                lstm.inputY: batchY, 
                lstm.dropoutKeepProb: config.model.dropoutKeepProb
            }
            _, summary, step, loss, predictions = sess.run(
            [trainOp, summaryOp, globalStep, lstm.loss, lstm.predictions], feed_dict)
            
            if config.numClasses == 1:
                acc, recall, prec, f_beta = get_binary_metrics(pred_y=predictions, true_y=batchY)
            elif config.numClasses > 1:
                acc, recall, prec, f_beta = get_multi_metrics(pred_y=predictions, true_y=batchY, labels=labelList)
            
            trainSummaryWriter.add_summary(summary, step)
            
            return loss, acc, prec, recall, f_beta
        
        
        def devStep(batchX, batchY):
            feed_dict = {
                lstm.inputX: batchX, 
                lstm.inputY: batchY, 
                lstm.dropoutKeepProb: 1.0
            }
            summary, step, loss, predictions = sess.run([summaryOp, globalStep, lstm.loss, lstm.predictions], feed_dict)
            
            if config.numClasses == 1:
                acc, precision, recall, f_beta = get_binary_metrics(pred_y=predictions, true_y=batchY)
            elif config.numClasses > 1:
                acc, precision, recall, f_beta = get_multi_metrics(pred_y=predictions, true_y=batchY, labels=labelList)
            
            evalSummaryWriter.add_summary(summary, step)
            
            return loss, acc, precision, recall, f_beta
        
        
        
        for index in range(config.training.epochs):
            for batchTrain in nextBatch(trainReviews, trainLabels, config.batchSize):
                loss, acc, prec, recall, f_beta = trainStep(batchTrain[0], batchTrain[1])
                currentStep = tf.train.global_step(sess, globalStep)
                print("train: step: {}, loss: {}, acc: {}, recall: {}, precision: {}, f_beta: {}".format(
                    currentStep, loss, acc, recall, prec, f_beta))
                
                if currentStep % config.training.evaluateEvery == 0:
                    print("begin to evaluation: ")
                    losses, accs, f_betas, precisions, recalls = [], [], [], [], []
                    for batchEval in nextBatch(evalReviews, evalLabels, config.batchSize):
                        loss, acc, precision, recall, f_beta = devStep(batchEval[0],batchEval[1])
                        losses.append(loss)
                        accs.append(acc)
                        f_betas.append(f_beta)
                        precisions.append(precision)
                        recalls.append(recall)
                    
                    time_str = datetime.datetime.now().isoformat()
                    print("{}, step: {}, loss: {}, acc: {}, precision: {}, recall: {}, f_beta: {}".format(
                            time_str, currentStep, mean(losses), mean(accs), mean(precisions), mean(recalls), mean(f_betas)))
            
                if currentStep % config.training.checkpointEvery == 0:
                    path = saver.save(sess, "\\".join(savedModelPath.split("\\")[:-1]) + "\\my_model", global_step=currentStep)
                    print("save model checkpoint to {}".format(path))
        
        
        inputs = {"inputX": tf.saved_model.utils.build_tensor_info(lstm.inputX),
                  "keepProb": tf.saved_model.utils.build_tensor_info(lstm.dropoutKeepProb)}

        outputs = {"predictions": tf.saved_model.utils.build_tensor_info(lstm.predictions)}

        prediction_signature = tf.saved_model.signature_def_utils.build_signature_def(inputs=inputs, outputs=outputs,
                                                                                      method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)
        legacy_init_op = tf.group(tf.tables_initializer(), name="legacy_init_op")
        builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING],
                                            signature_def_map={"predict": prediction_signature}, legacy_init_op=legacy_init_op)

        builder.save()    

the shape of fwOutput is:  Tensor("Shape:0", shape=(3,), dtype=int32)
the shape of bwOutput is:  Tensor("Shape_1:0", shape=(3,), dtype=int32)
Writing to C:\Users\123\Documents\python_experence\nlp_model\model_code\summarys

train: step: 1, loss: 1.139818787574768, acc: 0.5234375, recall: 1.0, precision: 0.5234375, f_beta: 0.6871794871794872
train: step: 2, loss: 0.9735010266304016, acc: 0.546875, recall: 1.0, precision: 0.546875, f_beta: 0.7070707070707071
train: step: 3, loss: 0.9532228708267212, acc: 0.484375, recall: 1.0, precision: 0.484375, f_beta: 0.6526315789473685
train: step: 4, loss: 0.7407537698745728, acc: 0.5859375, recall: 1.0, precision: 0.5859375, f_beta: 0.7389162561576355
train: step: 5, loss: 0.8292919993400574, acc: 0.46875, recall: 1.0, precision: 0.46875, f_beta: 0.6382978723404256
train: step: 6, loss: 0.8278107643127441, acc: 0.4375, recall: 1.0, precision: 0.4375, f_beta: 0.6086956521739131
train: step: 7, loss: 0.7091308236122131, acc: 0.5703125, recall: 1.0, 

W0924 18:08:26.377333 17960 deprecation.py:323] From d:\Users\123\Anaconda3\lib\site-packages\tensorflow\python\training\saver.py:960: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.


save model checkpoint to C:\Users\123\Documents\python_experence\nlp_model\model\textCNN\my_model-600
train: step: 601, loss: 0.22476764023303986, acc: 0.921875, recall: 0.9714285714285714, precision: 0.8947368421052632, f_beta: 0.9315068493150684
train: step: 602, loss: 0.19431564211845398, acc: 0.9453125, recall: 0.9565217391304348, precision: 0.9428571428571428, f_beta: 0.9496402877697843
train: step: 603, loss: 0.1824030727148056, acc: 0.9296875, recall: 0.9041095890410958, precision: 0.9705882352941176, f_beta: 0.9361702127659575
train: step: 604, loss: 0.17961876094341278, acc: 0.9453125, recall: 0.9615384615384616, precision: 0.9090909090909091, f_beta: 0.9345794392523366
train: step: 605, loss: 0.1651681512594223, acc: 0.953125, recall: 0.9523809523809523, precision: 0.9523809523809523, f_beta: 0.9523809523809523
train: step: 606, loss: 0.2922540307044983, acc: 0.8828125, recall: 0.9354838709677419, precision: 0.8405797101449275, f_beta: 0.8854961832061068
train: step: 607, los

W0924 18:50:37.097437 17960 deprecation.py:323] From <ipython-input-40-e1759701fd2b>:117: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


train: step: 1560, loss: 0.0052415719255805016, acc: 1.0, recall: 1.0, precision: 1.0, f_beta: 1.0


W0924 18:50:38.001839 17960 deprecation.py:506] From <ipython-input-40-e1759701fd2b>:126: calling SavedModelBuilder.add_meta_graph_and_variables (from tensorflow.python.saved_model.builder_impl) with legacy_init_op is deprecated and will be removed in a future version.
Instructions for updating:
Pass your op to the equivalent parameter main_op instead.
